In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### DataLoader and Dataset

In [3]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize,
    ])

    if test:
        dataset = datasets.CIFAR10(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(42)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


In [4]:
# CIFAR10 dataset
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

100%|██████████| 170498071/170498071 [00:03<00:00, 48302898.49it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


### GoogleNet Implementation

In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock,self).__init__()
        self.conv= nn.Conv2d(in_channels, out_channels, **kwargs)
        self.BatchNorm= nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    def forward(self,x):
        return self.relu(self.BatchNorm(self.conv(x)))

In [6]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels ,n1x1, reducer3x3, n3x3, reducer5x5, n5x5, pool_planes):
        super(InceptionBlock,self).__init__()
        self.oneIncetion = ConvBlock(in_channels, n1x1, kernel_size=1)
        self.threeInception = nn.Sequential(
            ConvBlock(in_channels, reducer3x3, kernel_size=1),
            ConvBlock(reducer3x3, n3x3, kernel_size=(3,3), padding=1)
        )
        self.fiveInception= nn.Sequential(
            ConvBlock(in_channels, reducer5x5, kernel_size=1),
            ConvBlock(reducer5x5, n5x5, kernel_size=5, padding=2)
        )
        self.poolInception= nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels, pool_planes, kernel_size=1)
        )
    def forward(self,x):
        return torch.cat([self.oneIncetion(x), self.threeInception(x), self.fiveInception(x), self.poolInception(x)],1)

In [26]:
class GoogLeNet(nn.Module):
    def __init__(self, in_channels, num_classes = 10):
        super(GoogLeNet, self).__init__()
        self.ConvLayer1 = nn.Sequential(
            ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ConvBlock(64, 192, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.ConvLayer2 = nn.Sequential(
            InceptionBlock(192, 64, 96, 128, 16, 32, 32),
            InceptionBlock(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        self.ConvLayer3= nn.Sequential(
            InceptionBlock(480, 192, 96, 208, 16, 48, 64),
            InceptionBlock(512, 160, 112, 224, 24, 64, 64),
            InceptionBlock(512, 128, 128, 256, 24, 64, 64),
            InceptionBlock(512, 112, 144, 288, 32, 64, 64),
            InceptionBlock(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        self.ConvLayer4= nn.Sequential(
            InceptionBlock(832, 256, 160, 320, 32, 128, 128),
            InceptionBlock(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(kernel_size=7, stride= 1),
        )
        self.fc= nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(1024, num_classes)
        )
    def forward(self,x):
        x= self.ConvLayer1(x)
        #print(x.size())
        x = self.ConvLayer2(x)
        #print(x.size())
        x = self.ConvLayer3(x)
        #print(x.size())
        x = self.ConvLayer4(x)
        #print(x.size())
        x= x.reshape(x.shape[0],-1)
        #print(x.size())
        return self.fc(x)

### ResNet Implementation

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

### Train

In [9]:
import gc
total_step = len(train_loader)

def train(model, criterion, optimizer):
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of validation images: {} %'.format(100 * correct / total))

### Test

In [10]:
def test(model):
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))

### GoogLeNet training and test

In [11]:
num_epochs = 2
learning_rate = 0.01

In [27]:
model_g = GoogLeNet(3).to(device)

criterion_g = nn.CrossEntropyLoss()
optimizer_g = torch.optim.SGD(model_g.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

In [28]:
train(model_g, criterion_g, optimizer_g)

Epoch [1/2], Loss: 1.3748
Accuracy of validation images: 61.06 %
Epoch [2/2], Loss: 1.5713
Accuracy of validation images: 73.38 %


In [21]:
test(model_g)

Accuracy of the network on the test images: 71.27 %


### ResNet50 training and test

In [ ]:
model_res = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion_res = nn.CrossEntropyLoss()
optimizer_res = torch.optim.SGD(model_res.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

In [ ]:
train(model_res, criterion_res, optimizer_res)

Epoch [1/2], Loss: 1.3558
Accuracy of validation images: 55.94 %
Epoch [2/2], Loss: 1.7125
Accuracy of validation images: 70.88 %


In [ ]:
test(model_res)

Accuracy of the network on the test images: 70.2 %
